In [1]:
#load packages
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import soundfile as sf


In [2]:
#import data 
path = 'x:/Steering/Recordings/Processed/'
data1,_ = sf.read(path + 'ERB=2_MF_01.wav')
data2,_ = sf.read(path + 'ERB=2_MF_02.wav')
data3,_ = sf.read(path + 'ERB=2_MF_03.wav')

classes = pd.read_csv(path + 'Classes_2_MF.csv')

data = np.hstack([data1, data2, data3])

In [3]:
data.shape

(970584, 756)

In [4]:
X = data[:len(data)-1,:data.shape[1]] #775361
Y = classes.values[:,0]

In [5]:
X.shape, Y.shape

((970583, 756), (970583,))

In [6]:
#split into training and test set
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.2,random_state=0)

In [7]:
#do one-hot encoding
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder

enc = OneHotEncoder(sparse=False)

In [8]:
integer_encoded_train = trainY.reshape(len(trainY), 1)
onehot_encoded_train = enc.fit_transform(integer_encoded_train)

integer_encoded_test = testY.reshape(len(testY), 1)
onehot_encoded_test = enc.fit_transform(integer_encoded_test)

In [9]:
onehot_encoded_train 


array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [10]:
#DC removal 
trainX -= (np.mean(trainX, axis=0) + 1e-8)
testX -= (np.mean(testX, axis=0) + 1e-8)

In [11]:
clf = keras.Sequential()

In [12]:
trainX.shape, onehot_encoded_train.shape

((776466, 756), (776466, 5))

In [13]:
clf.add(keras.layers.Dense(
  units = 18,
  input_dim = trainX.shape[1],   
  activation = 'tanh'
))

clf.add (keras.layers.Dense(
    units = 7,
    input_dim = 18,
    activation = 'tanh'
))

clf.add(keras.layers.Dense(
    units = onehot_encoded_train.shape[1],
    input_dim = 7,
    activation = 'softmax'
))


In [14]:
onehot_encoded_train.shape[1]

5

In [15]:
clf.compile(loss='mean_squared_error',
    optimizer='Adam',
           metrics=['categorical_accuracy'])

In [16]:
history = clf.fit(trainX,onehot_encoded_train,
                 batch_size = 64, epochs = 20,
                 verbose = 1,
                 validation_split = 0.1)

Train on 698819 samples, validate on 77647 samples
Epoch 1/20
698819/698819 [==============================] - 18s 26us/step - loss: 0.0478 - categorical_accuracy: 0.8408 - val_loss: 0.0246 - val_categorical_accuracy: 0.9192
Epoch 2/20
698819/698819 [==============================] - 15s 22us/step - loss: 0.0179 - categorical_accuracy: 0.9455 - val_loss: 0.0131 - val_categorical_accuracy: 0.9652
Epoch 3/20
698819/698819 [==============================] - 13s 19us/step - loss: 0.0108 - categorical_accuracy: 0.9679 - val_loss: 0.0145 - val_categorical_accuracy: 0.9531
Epoch 4/20
698819/698819 [==============================] - 14s 20us/step - loss: 0.0085 - categorical_accuracy: 0.9745 - val_loss: 0.0069 - val_categorical_accuracy: 0.9806
Epoch 5/20
698819/698819 [==============================] - 14s 20us/step - loss: 0.0076 - categorical_accuracy: 0.9766 - val_loss: 0.0072 - val_categorical_accuracy: 0.9796
Epoch 6/20
698819/698819 [==============================] - 20s 28us/step - los

In [17]:
data.shape[1]

756

In [18]:
clf.predict_classes(testX[1:100,:]) + 1,testY[:100]

(array([2, 5, 1, 5, 2, 4, 4, 2, 4, 4, 3, 2, 3, 1, 3, 3, 2, 4, 3, 4, 2, 4,
        4, 2, 5, 5, 4, 5, 2, 2, 5, 4, 4, 1, 5, 4, 2, 2, 5, 2, 4, 4, 5, 3,
        1, 3, 5, 5, 3, 2, 2, 5, 1, 5, 5, 4, 5, 4, 4, 2, 2, 2, 1, 4, 2, 1,
        2, 2, 1, 4, 2, 1, 4, 2, 1, 5, 3, 4, 5, 4, 3, 5, 2, 5, 2, 2, 2, 2,
        5, 5, 5, 4, 4, 2, 4, 5, 2, 1, 1], dtype=int64),
 array([2, 2, 5, 1, 5, 2, 4, 4, 2, 4, 4, 3, 2, 3, 1, 3, 3, 2, 4, 3, 4, 2,
        4, 4, 2, 5, 5, 4, 5, 2, 4, 5, 4, 4, 1, 5, 4, 2, 2, 5, 2, 4, 4, 5,
        3, 1, 3, 5, 5, 3, 2, 4, 5, 1, 5, 5, 4, 5, 4, 4, 2, 2, 2, 1, 4, 2,
        1, 2, 2, 1, 4, 2, 1, 4, 2, 1, 5, 3, 4, 5, 4, 3, 5, 2, 5, 4, 2, 2,
        2, 4, 5, 5, 4, 4, 2, 4, 5, 2, 1, 1], dtype=int64))

In [19]:
# saving the model 
# save model and weights
clf_json = clf.to_json()
with open("NN_756-18-7-5.json", "w") as json_file:
    json_file.write(clf_json)
    
clf.save_weights("NN_756-18-7-5_weights.h5")

In [20]:
np.save('TestX_NN_756-18-7-5',testX), np.save('TestY_NN_756-18-7-5',testY),
np.save('OneHot_testY_756-18-7-5',onehot_encoded_test)

In [22]:
clf.evaluate(testX,onehot_encoded_test)

194117/194117 [==============================] - 3s 14us/step


[0.005279907120534479, 0.9826702452644539]